In [1]:
import cdsapi
import getpass
import glob
import os
    
from azure.storage.blob import BlockBlobService
from calendar import monthrange

In [2]:
key = getpass.getpass()

········


In [2]:
llon = 4.5
ulon = 32
llat = 53.5
ulat = 71.5

In [4]:
blob_storage = BlockBlobService(account_name="ccclimatedata", account_key=key)
blob_storage_container_name = "era5-nordics-temperature"

# Retrieve temperatures at 2 meters (t2m and d2m) from ECMWF CDS

In [11]:
for year in range(2018, 2019):
    for month in range(1, 13):
        days = range(1, monthrange(year, month)[1] + 1)

        request = {
                'variable':[
                    '2m_dewpoint_temperature','2m_temperature'
                ],
                'product_type':'reanalysis',
                'year': "%04d" % year,
                'month': "%02d" % month ,
                'day': ["%02d" % d for d in range(1, 32)],
                'time': ["%02d:00" % h for h in range(0, 24)],
                'area': [
                    ulat, llon, llat, ulon # North, West, South, East
                ],
                'format':'netcdf'
            }

        c = cdsapi.Client()

        r = c.retrieve(
            'reanalysis-era5-single-levels',
            request
            )

        file_path = "era5-nordics-temperature-%04d-%02d.nc" % (year, month)

        r.download(file_path)

        #blob_storage.create_blob_from_path(
        #    container_name=blob_storage_container_name,
        #    file_path=file_path,
        #    blob_name=file_path,
        #    timeout=360
        #)

2018-11-19 20:43:47,975 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2018-11-19 20:43:49,296 INFO Request is queued
2018-11-19 20:43:52,400 INFO Request is running
2018-11-19 20:45:45,608 INFO Request is completed
2018-11-19 20:45:46,934 INFO Downloading http://136.156.132.110/cache-compute-0001/cache/data1/adaptor.mars.internal-1542667432.960177-713-3-d88af291-310f-429e-a1bb-049fa38a86c0.nc to era5-nordics-temperature-2018-01.nc (23M)
2018-11-19 20:46:42,551 INFO Download rate 423.5K/s
2018-11-19 20:46:42,555 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2018-11-19 20:46:43,634 INFO Request is queued
2018-11-19 20:46:46,672 INFO Request is running
2018-11-19 20:48:01,212 INFO Request is completed
2018-11-19 20:48:02,493 INFO Downloading http://136.156.132.110/cache-compute-0002/cache/data2/adaptor.mars.internal-1542667608.0107353-24556-15-f1dea694-f4d8-4235-a9c2-41c30

Exception: no data is available within your requested subset. Request returned no data.

# Temperature at 2m - Hourly climatology

Use CDO to calculate min, mean and maximum values at horly scale for the entire interval

In [8]:
!cdo -b 32 yhourmean -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-200*-*.nc' era5-nordics-t2m_hourly.nc

cdo yhourmean: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-200*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-200*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-200*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(4) cat: Processed 1420812432 values from 240 variables over 87672 timesteps [16.15s]
cdo(3) select: Processed 710406216 values from 2 variables over 87672 timesteps [16.64s]
cdo(2) addc: Processed 710406216 values from 1 variable over 87672 timesteps [16.64s]
cdo yhourmean: Processed 710406216 values from 1 variable over 87672 timesteps [16.64s 595MB]


Inspect the results

In [9]:
!cdo info era5-nordics-t2m_hourly.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2009-01-01 00:00:00       0     8103       0 :     -17.574     -4.3087      6.0441 : -1            
     2 : 2009-01-01 01:00:00       0     8103       0 :     -17.685     -4.3315      6.0436 : -1            
     3 : 2009-01-01 02:00:00       0     8103       0 :     -17.683     -4.3249      6.0641 : -1            
     4 : 2009-01-01 03:00:00       0     8103       0 :     -17.653     -4.3039      6.0384 : -1            
     5 : 2009-01-01 04:00:00       0     8103       0 :     -17.611     -4.2988      5.9648 : -1            
     6 : 2009-01-01 05:00:00       0     8103       0 :     -17.487     -4.3022      5.9157 : -1            
     7 : 2009-01-01 06:00:00       0     8103       0 :     -17.227     -4.2759      6.0639 : -1            
     8 : 2009-01-01 07:00:00       0     8103       0 :     -17.343     -4.2818      6.1767 : -1            
     9 : 2009-01-01 0

   370 : 2009-01-16 09:00:00       0     8103       0 :     -11.014     -1.0766      6.5388 : -1            
   371 : 2009-01-16 10:00:00       0     8103       0 :     -10.706    -0.81994      6.4267 : -1            
   372 : 2009-01-16 11:00:00       0     8103       0 :     -10.535    -0.73680      6.4612 : -1            
   373 : 2009-01-16 12:00:00       0     8103       0 :     -10.706    -0.68038      6.4987 : -1            
   374 : 2009-01-16 13:00:00       0     8103       0 :     -10.720    -0.65280      6.4765 : -1            
   375 : 2009-01-16 14:00:00       0     8103       0 :     -10.783    -0.75022      6.4974 : -1            
   376 : 2009-01-16 15:00:00       0     8103       0 :     -10.868    -0.83152      6.4697 : -1            
   377 : 2009-01-16 16:00:00       0     8103       0 :     -10.588    -0.85578      6.4422 : -1            
   378 : 2009-01-16 17:00:00       0     8103       0 :     -10.552    -0.97632      6.4185 : -1            
   379 : 2009-01-16

  1320 : 2009-02-24 23:00:00       0     8103       0 :     -13.727     -4.1014      5.0923 : -1            
  1321 : 2009-02-25 00:00:00       0     8103       0 :     -13.713     -4.1684      5.0044 : -1            
  1322 : 2009-02-25 01:00:00       0     8103       0 :     -13.618     -4.2146      4.9385 : -1            
  1323 : 2009-02-25 02:00:00       0     8103       0 :     -13.836     -4.3810      4.9088 : -1            
  1324 : 2009-02-25 03:00:00       0     8103       0 :     -13.954     -4.3921      4.8879 : -1            
  1325 : 2009-02-25 04:00:00       0     8103       0 :     -13.951     -4.4339      4.8517 : -1            
  1326 : 2009-02-25 05:00:00       0     8103       0 :     -14.316     -4.5390      4.7630 : -1            
  1327 : 2009-02-25 06:00:00       0     8103       0 :     -14.230     -4.5268      4.7517 : -1            
  1328 : 2009-02-25 07:00:00       0     8103       0 :     -14.182     -4.4516      4.8324 : -1            
  1329 : 2009-02-25

  2280 : 2009-04-04 23:00:00       0     8103       0 :     -9.6215     0.74996      7.1532 : -1            
  2281 : 2009-04-05 00:00:00       0     8103       0 :     -9.5237     0.65187      7.0760 : -1            
  2282 : 2009-04-05 01:00:00       0     8103       0 :     -9.4832     0.55284      6.9297 : -1            
  2283 : 2009-04-05 02:00:00       0     8103       0 :     -9.4505     0.30583      6.7688 : -1            
  2284 : 2009-04-05 03:00:00       0     8103       0 :     -9.3419     0.28207      6.6559 : -1            
  2285 : 2009-04-05 04:00:00       0     8103       0 :     -9.3473     0.22181      6.5651 : -1            
  2286 : 2009-04-05 05:00:00       0     8103       0 :     -8.9591     0.62896      6.4806 : -1            
  2287 : 2009-04-05 06:00:00       0     8103       0 :     -8.6653     0.85627      6.4439 : -1            
  2288 : 2009-04-05 07:00:00       0     8103       0 :     -8.5401      1.1114      6.4585 : -1            
  2289 : 2009-04-05

  3371 : 2009-05-20 10:00:00       0     8103       0 :      1.0544      9.8618      17.564 : -1            
  3372 : 2009-05-20 11:00:00       0     8103       0 :      1.0130      9.9756      17.415 : -1            
  3373 : 2009-05-20 12:00:00       0     8103       0 :      1.1551      10.066      17.810 : -1            
  3374 : 2009-05-20 13:00:00       0     8103       0 :      1.2200      10.132      17.729 : -1            
  3375 : 2009-05-20 14:00:00       0     8103       0 :      1.4881      10.181      17.677 : -1            
  3376 : 2009-05-20 15:00:00       0     8103       0 :      1.5058      10.064      17.374 : -1            
  3377 : 2009-05-20 16:00:00       0     8103       0 :      1.6693      10.000      17.031 : -1            
  3378 : 2009-05-20 17:00:00       0     8103       0 :      1.4457      9.3995      16.215 : -1            
  3379 : 2009-05-20 18:00:00       0     8103       0 :      1.4328      9.1885      15.241 : -1            
  3380 : 2009-05-20

  4352 : 2009-06-30 07:00:00       0     8103       0 :      6.3243      13.375      17.920 : -1            
  4353 : 2009-06-30 08:00:00       0     8103       0 :      6.3986      14.331      19.805 : -1            
  4354 : 2009-06-30 09:00:00       0     8103       0 :      6.4688      14.713      20.494 : -1            
  4355 : 2009-06-30 10:00:00       0     8103       0 :      6.6251      15.462      21.016 : -1            
  4356 : 2009-06-30 11:00:00       0     8103       0 :      6.6984      15.626      21.166 : -1            
  4357 : 2009-06-30 12:00:00       0     8103       0 :      6.7485      15.760      21.466 : -1            
  4358 : 2009-06-30 13:00:00       0     8103       0 :      6.7850      15.835      21.852 : -1            
  4359 : 2009-06-30 14:00:00       0     8103       0 :      6.7997      15.903      22.192 : -1            
  4360 : 2009-06-30 15:00:00       0     8103       0 :      6.8013      15.846      22.211 : -1            
  4361 : 2009-06-30

  5317 : 2009-08-09 12:00:00       0     8103       0 :      8.7224      17.403      23.626 : -1            
  5318 : 2009-08-09 13:00:00       0     8103       0 :      8.7152      17.456      23.691 : -1            
  5319 : 2009-08-09 14:00:00       0     8103       0 :      8.6938      17.400      23.689 : -1            
  5320 : 2009-08-09 15:00:00       0     8103       0 :      8.6858      17.238      23.354 : -1            
  5321 : 2009-08-09 16:00:00       0     8103       0 :      8.6903      17.141      22.850 : -1            
  5322 : 2009-08-09 17:00:00       0     8103       0 :      8.6666      16.294      22.352 : -1            
  5323 : 2009-08-09 18:00:00       0     8103       0 :      8.6217      16.011      21.041 : -1            
  5324 : 2009-08-09 19:00:00       0     8103       0 :      8.5717      15.676      20.669 : -1            
  5325 : 2009-08-09 20:00:00       0     8103       0 :      8.5104      14.454      20.458 : -1            
  5326 : 2009-08-09

  6212 : 2009-09-15 19:00:00       0     8103       0 :      2.8640      9.7818      16.333 : -1            
  6213 : 2009-09-15 20:00:00       0     8103       0 :      2.4968      9.2387      16.242 : -1            
  6214 : 2009-09-15 21:00:00       0     8103       0 :      1.4958      8.9946      16.150 : -1            
  6215 : 2009-09-15 22:00:00       0     8103       0 :     0.95070      8.5331      15.973 : -1            
  6216 : 2009-09-15 23:00:00       0     8103       0 :     0.77793      8.4406      15.892 : -1            
  6217 : 2009-09-16 00:00:00       0     8103       0 :     0.68009      8.3481      15.833 : -1            
  6218 : 2009-09-16 01:00:00       0     8103       0 :     0.46434      8.2620      15.774 : -1            
  6219 : 2009-09-16 02:00:00       0     8103       0 :     0.38718      8.0387      15.727 : -1            
  6220 : 2009-09-16 03:00:00       0     8103       0 :     0.58245      8.0411      15.703 : -1            
  6221 : 2009-09-16

  7135 : 2009-10-24 06:00:00       0     8103       0 :     -6.5997      3.3060      12.436 : -1            
  7136 : 2009-10-24 07:00:00       0     8103       0 :     -6.4287      3.3770      12.496 : -1            
  7137 : 2009-10-24 08:00:00       0     8103       0 :     -5.6022      4.0084      12.599 : -1            
  7138 : 2009-10-24 09:00:00       0     8103       0 :     -5.3737      4.2347      12.668 : -1            
  7139 : 2009-10-24 10:00:00       0     8103       0 :     -4.6122      4.8770      12.523 : -1            
  7140 : 2009-10-24 11:00:00       0     8103       0 :     -4.3996      5.0001      12.425 : -1            
  7141 : 2009-10-24 12:00:00       0     8103       0 :     -4.3983      5.0654      12.361 : -1            
  7142 : 2009-10-24 13:00:00       0     8103       0 :     -4.4538      5.0891      12.359 : -1            
  7143 : 2009-10-24 14:00:00       0     8103       0 :     -4.9676      4.8738      12.599 : -1            
  7144 : 2009-10-24

  8073 : 2009-12-02 08:00:00       0     8103       0 :     -10.097     0.54470      8.9799 : -1            
  8074 : 2009-12-02 09:00:00       0     8103       0 :     -10.183     0.59794      9.0131 : -1            
  8075 : 2009-12-02 10:00:00       0     8103       0 :     -10.524     0.80837      8.9735 : -1            
  8076 : 2009-12-02 11:00:00       0     8103       0 :     -10.599     0.86364      8.9142 : -1            
  8077 : 2009-12-02 12:00:00       0     8103       0 :     -10.521     0.88617      8.8566 : -1            
  8078 : 2009-12-02 13:00:00       0     8103       0 :     -10.560     0.86950      8.8622 : -1            
  8079 : 2009-12-02 14:00:00       0     8103       0 :     -10.747     0.76069      8.8899 : -1            
  8080 : 2009-12-02 15:00:00       0     8103       0 :     -10.742     0.67675      8.9663 : -1            
  8081 : 2009-12-02 16:00:00       0     8103       0 :     -10.497     0.65740      8.9991 : -1            
  8082 : 2009-12-02

Upload result to Blob Storage

In [ ]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-t2m_hourly.nc",
    blob_name="era5-nordics-t2m_hourly.nc",
    timeout=180
)

In [1]:
!cdo -b 32 ymonmean -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_ymon.nc

cdo ymonmean: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(4) cat: Processed 2651820192 values from 448 variables over 163632 timesteps [88.88s]
cdo(3) select: Processed 1325910096 values from 2 variables over 163632 timesteps [88.88s]
cdo(2) addc: Processed 1325910096 values from 1 variable over 163632 timesteps [88.88s]
cdo ymonmean: Processed 1325910096 values from 1 variable over 163632 timesteps [88.88s 47MB]


In [2]:
!cdo -b 32 ymonstd -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_ymonstd.nc

cdo ymonstd: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(4) cat: Processed 2651820192 values from 448 variables over 163632 timesteps [87.12s]
cdo(3) select: Processed 1325910096 values from 2 variables over 163632 timesteps [87.13s]
cdo(2) addc: Processed 1325910096 values from 1 variable over 163632 timesteps [87.13s]
cdo ymonstd: Processed 1325910096 values from 1 variable over 163632 timesteps [87.13s 48MB]


# Temperature at 2m - Daily Climatology

In [2]:
!cdo -b 32 daymean -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_daily.nc

cdo daymean: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(4) cat: Processed 2651820192 values from 448 variables over 163632 timesteps [39.78s]
cdo(3) select: Processed 1325910096 values from 2 variables over 163632 timesteps [39.79s]
cdo(2) addc: Processed 1325910096 values from 1 variable over 163632 timesteps [39.79s]
cdo daymean: Processed 1325910096 values from 1 variable over 163632 timesteps [39.79s 48MB]


In [3]:
!cdo info era5-nordics-t2m_daily.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2000-01-01 11:30:00       0     8103       0 :     -15.508     -3.5047      7.7659 : -1            
     2 : 2000-01-02 11:30:00       0     8103       0 :     -14.672     -1.4637      7.8487 : -1            
     3 : 2000-01-03 11:30:00       0     8103       0 :     -12.939    -0.29444      8.6798 : -1            
     4 : 2000-01-04 11:30:00       0     8103       0 :     -11.305     0.31955      7.1163 : -1            
     5 : 2000-01-05 11:30:00       0     8103       0 :     -14.721     -1.2816      7.3423 : -1            
     6 : 2000-01-06 11:30:00       0     8103       0 :     -13.735     0.20619      8.4529 : -1            
     7 : 2000-01-07 11:30:00       0     8103       0 :     -9.0500      1.7509      7.9701 : -1            
     8 : 2000-01-08 11:30:00       0     8103       0 :     -6.7797      2.8695      7.9049 : -1            
     9 : 2000-01-09 1

   573 : 2001-07-26 11:30:00       0     8103       0 :      7.4081      16.013      24.618 : -1            
   574 : 2001-07-27 11:30:00       0     8103       0 :      4.6402      15.352      23.223 : -1            
   575 : 2001-07-28 11:30:00       0     8103       0 :      3.5191      14.833      22.748 : -1            
   576 : 2001-07-29 11:30:00       0     8103       0 :      6.4209      15.531      22.491 : -1            
   577 : 2001-07-30 11:30:00       0     8103       0 :      3.9161      15.156      22.720 : -1            
   578 : 2001-07-31 11:30:00       0     8103       0 :      4.0154      13.067      22.257 : -1            
   579 : 2001-08-01 11:30:00       0     8103       0 :      1.1148      11.729      18.470 : -1            
   580 : 2001-08-02 11:30:00       0     8103       0 :      1.2911      11.443      18.696 : -1            
   581 : 2001-08-03 11:30:00       0     8103       0 :      3.4772      12.688      21.390 : -1            
   582 : 2001-08-04

  1580 : 2004-04-28 11:30:00       0     8103       0 :     -1.7369      5.8524      14.539 : -1            
  1581 : 2004-04-29 11:30:00       0     8103       0 :     -1.7952      5.9608      13.204 : -1            
  1582 : 2004-04-30 11:30:00       0     8103       0 :     -1.0526      5.9968      15.208 : -1            
  1583 : 2004-05-01 11:30:00       0     8103       0 :     -1.6880      6.7295      16.489 : -1            
  1584 : 2004-05-02 11:30:00       0     8103       0 :     0.26670      7.8991      15.059 : -1            
  1585 : 2004-05-03 11:30:00       0     8103       0 :     0.20436      9.1019      16.045 : -1            
  1586 : 2004-05-04 11:30:00       0     8103       0 :    -0.29701      9.6730      17.745 : -1            
  1587 : 2004-05-05 11:30:00       0     8103       0 :      1.7339      11.365      20.985 : -1            
  1588 : 2004-05-06 11:30:00       0     8103       0 :      2.5159      12.469      20.935 : -1            
  1589 : 2004-05-07

  2579 : 2007-01-22 11:30:00       0     8103       0 :     -27.663     -7.4185      5.7934 : -1            
  2580 : 2007-01-23 11:30:00       0     8103       0 :     -30.137     -8.1844      4.8866 : -1            
  2581 : 2007-01-24 11:30:00       0     8103       0 :     -28.774     -8.5563      4.4860 : -1            
  2582 : 2007-01-25 11:30:00       0     8103       0 :     -20.781     -5.4743      6.1038 : -1            
  2583 : 2007-01-26 11:30:00       0     8103       0 :     -17.788     -3.6833      6.5428 : -1            
  2584 : 2007-01-27 11:30:00       0     8103       0 :     -18.608     -5.4897      6.3156 : -1            
  2585 : 2007-01-28 11:30:00       0     8103       0 :     -24.374     -6.6012      8.1234 : -1            
  2586 : 2007-01-29 11:30:00       0     8103       0 :     -29.772     -6.3805      8.2267 : -1            
  2587 : 2007-01-30 11:30:00       0     8103       0 :     -30.193     -5.6069      7.9471 : -1            
  2588 : 2007-01-31

  3129 : 2008-07-25 11:30:00       0     8103       0 :      7.8108      16.674      24.574 : -1            
  3130 : 2008-07-26 11:30:00       0     8103       0 :      6.6904      16.757      24.694 : -1            
  3131 : 2008-07-27 11:30:00       0     8103       0 :      5.9443      16.384      24.211 : -1            
  3132 : 2008-07-28 11:30:00       0     8103       0 :      6.8657      15.903      25.325 : -1            
  3133 : 2008-07-29 11:30:00       0     8103       0 :      8.4828      15.757      25.254 : -1            
  3134 : 2008-07-30 11:30:00       0     8103       0 :      7.5695      15.884      22.361 : -1            
  3135 : 2008-07-31 11:30:00       0     8103       0 :      4.9771      15.481      23.743 : -1            
  3136 : 2008-08-01 11:30:00       0     8103       0 :      4.0060      14.569      23.908 : -1            
  3137 : 2008-08-02 11:30:00       0     8103       0 :      3.8865      13.639      22.358 : -1            
  3138 : 2008-08-03

  4123 : 2011-04-15 11:30:00       0     8103       0 :     -1.0010      4.7426      9.0848 : -1            
  4124 : 2011-04-16 11:30:00       0     8103       0 :     -2.6981      5.2190      11.268 : -1            
  4125 : 2011-04-17 11:30:00       0     8103       0 :     -3.4791      5.6725      12.341 : -1            
  4126 : 2011-04-18 11:30:00       0     8103       0 :     -5.1938      4.8338      12.844 : -1            
  4127 : 2011-04-19 11:30:00       0     8103       0 :     -2.6221      4.4916      14.542 : -1            
  4128 : 2011-04-20 11:30:00       0     8103       0 :     -1.8262      4.6676      15.335 : -1            
  4129 : 2011-04-21 11:30:00       0     8103       0 :     -2.4012      5.6668      16.362 : -1            
  4130 : 2011-04-22 11:30:00       0     8103       0 :     -1.6805      6.5369      16.640 : -1            
  4131 : 2011-04-23 11:30:00       0     8103       0 :     -2.7008      7.2157      17.096 : -1            
  4132 : 2011-04-24

  5161 : 2014-02-16 11:30:00       0     8103       0 :     -5.7907      1.7235      7.0253 : -1            
  5162 : 2014-02-17 11:30:00       0     8103       0 :     -9.0425      1.6002      7.1540 : -1            
  5163 : 2014-02-18 11:30:00       0     8103       0 :     -15.979     0.56696      7.3374 : -1            
  5164 : 2014-02-19 11:30:00       0     8103       0 :     -17.543    -0.90809      7.2596 : -1            
  5165 : 2014-02-20 11:30:00       0     8103       0 :     -15.764     -2.1839      8.0882 : -1            
  5166 : 2014-02-21 11:30:00       0     8103       0 :     -17.683     -1.6701      7.3042 : -1            
  5167 : 2014-02-22 11:30:00       0     8103       0 :     -11.752     0.64723      7.4793 : -1            
  5168 : 2014-02-23 11:30:00       0     8103       0 :     -7.8457      2.5620      8.2796 : -1            
  5169 : 2014-02-24 11:30:00       0     8103       0 :     -1.2640      4.0290      8.0753 : -1            
  5170 : 2014-02-25

  6145 : 2016-10-27 11:30:00       0     8103       0 :     -4.3188      5.1153      14.083 : -1            
  6146 : 2016-10-28 11:30:00       0     8103       0 :     -5.3182      5.7310      13.681 : -1            
  6147 : 2016-10-29 11:30:00       0     8103       0 :     -7.3768      4.8775      13.030 : -1            
  6148 : 2016-10-30 11:30:00       0     8103       0 :     -4.9004      4.3418      13.774 : -1            
  6149 : 2016-10-31 11:30:00       0     8103       0 :     -8.8946      3.6886      13.614 : -1            
  6150 : 2016-11-01 11:30:00       0     8103       0 :     -7.2004      2.9348      12.293 : -1            
  6151 : 2016-11-02 11:30:00       0     8103       0 :     -9.5529      1.7243      10.843 : -1            
  6152 : 2016-11-03 11:30:00       0     8103       0 :     -14.275     0.52676      11.130 : -1            
  6153 : 2016-11-04 11:30:00       0     8103       0 :     -13.149    -0.50510      10.823 : -1            
  6154 : 2016-11-05

In [5]:
!ncdump -h era5-nordics-t2m_daily.nc

netcdf era5-nordics-t2m_daily {
dimensions:
	time = UNLIMITED ; // (6818 currently)
	bnds = 2 ;
	longitude = 111 ;
	latitude = 73 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:long_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "hours since 1900-01-01 00:00:0.0" ;
		time:calendar = "gregorian" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	float longitude(longitude) ;
		longitude:standard_name = "longitude" ;
		longitude:long_name = "longitude" ;
		longitude:units = "degrees_east" ;
		longitude:axis = "X" ;
	float latitude(latitude) ;
		latitude:standard_name = "latitude" ;
		latitude:long_name = "latitude" ;
		latitude:units = "degrees_north" ;
		latitude:axis = "Y" ;
	float t2m(time, latitude, longitude) ;
		t2m:long_name = "2 metre temperature" ;
		t2m:units = "K" ;
		t2m:_FillValue = -32767.f ;
		t2m:missing_value = -32767.f ;

// global attributes:
		:CDI = "Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)" ;
		:Conventi

In [4]:
!cdo -b 32 ydaymean -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_ydaily.nc

cdo ydaymean: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(4) cat: Processed 2651820192 values from 448 variables over 163632 timesteps [86.20s]
cdo(3) select: Processed 1325910096 values from 2 variables over 163632 timesteps [86.22s]
cdo(2) addc: Processed 1325910096 values from 1 variable over 163632 timesteps [86.22s]
cdo ydaymean: Processed 1325910096 values from 1 variable over 163632 timesteps [86.22s 70MB]


In [6]:
!cdo info era5-nordics-t2m_ydaily.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2018-01-01 23:00:00       0     8103       0 :     -12.632     -2.4737      6.6705 : -1            
     2 : 2018-01-02 23:00:00       0     8103       0 :     -11.818     -2.8472      6.4030 : -1            
     3 : 2018-01-03 23:00:00       0     8103       0 :     -11.860     -3.0213      6.2231 : -1            
     4 : 2018-01-04 23:00:00       0     8103       0 :     -13.839     -3.4590      6.3471 : -1            
     5 : 2018-01-05 23:00:00       0     8103       0 :     -14.906     -4.0970      6.1612 : -1            
     6 : 2018-01-06 23:00:00       0     8103       0 :     -15.590     -4.1005      6.1493 : -1            
     7 : 2018-01-07 23:00:00       0     8103       0 :     -13.807     -3.2771      6.4766 : -1            
     8 : 2018-01-08 23:00:00       0     8103       0 :     -12.168     -2.2405      6.6863 : -1            
     9 : 2018-01-09 2

   105 : 2018-04-14 23:00:00       0     8103       0 :     -4.9648      3.1855      8.9556 : -1            
   106 : 2018-04-15 23:00:00       0     8103       0 :     -4.2793      3.5725      9.9380 : -1            
   107 : 2018-04-16 23:00:00       0     8103       0 :     -3.9838      3.8487      9.8450 : -1            
   108 : 2018-04-17 23:00:00       0     8103       0 :     -4.2783      3.8295      9.2864 : -1            
   109 : 2018-04-18 23:00:00       0     8103       0 :     -4.3282      3.6926      9.1925 : -1            
   110 : 2018-04-19 23:00:00       0     8103       0 :     -3.7211      3.9081      9.2550 : -1            
   111 : 2018-04-20 23:00:00       0     8103       0 :     -3.4032      4.0535      9.6686 : -1            
   112 : 2018-04-21 23:00:00       0     8103       0 :     -2.9303      4.3766      10.296 : -1            
   113 : 2018-04-22 23:00:00       0     8103       0 :     -2.8640      4.4945      10.465 : -1            
   114 : 2018-04-23

In [ ]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-t2m_daily.nc",
    blob_name="era5-nordics-t2m_daily.nc",
    timeout=180
)

Freezing degree days

In [10]:
!cdo -b 32 daymin -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_k_daily_min.nc

cdo daymin: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2651820192 values from 448 variables over 163632 timesteps [25.20s]
cdo(2) select: Processed 1325910096 values from 2 variables over 163632 timesteps [25.20s]
cdo daymin: Processed 1325910096 values from 1 variable over 163632 timesteps [25.20s 48MB]


In [11]:
!cdo info era5-nordics-t2m_k_daily_min.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2000-01-01 11:30:00       0     8103       0 :      250.73      267.69      280.60 : -1            
     2 : 2000-01-02 11:30:00       0     8103       0 :      255.56      269.30      280.56 : -1            
     3 : 2000-01-03 11:30:00       0     8103       0 :      258.33      270.95      280.79 : -1            
     4 : 2000-01-04 11:30:00       0     8103       0 :      256.70      271.52      279.80 : -1            
     5 : 2000-01-05 11:30:00       0     8103       0 :      249.52      269.47      279.98 : -1            
     6 : 2000-01-06 11:30:00       0     8103       0 :      250.64      270.11      280.55 : -1            
     7 : 2000-01-07 11:30:00       0     8103       0 :      261.10      272.99      280.35 : -1            
     8 : 2000-01-08 11:30:00       0     8103       0 :      263.21      274.66      280.35 : -1            
     9 : 2000-01-09 1

   163 : 2000-06-11 11:30:00       0     8103       0 :      271.34      282.34      289.52 : -1            
   164 : 2000-06-12 11:30:00       0     8103       0 :      272.58      281.53      287.51 : -1            
   165 : 2000-06-13 11:30:00       0     8103       0 :      272.28      281.52      288.89 : -1            
   166 : 2000-06-14 11:30:00       0     8103       0 :      271.29      281.08      288.16 : -1            
   167 : 2000-06-15 11:30:00       0     8103       0 :      269.63      279.64      286.09 : -1            
   168 : 2000-06-16 11:30:00       0     8103       0 :      268.12      278.87      285.11 : -1            
   169 : 2000-06-17 11:30:00       0     8103       0 :      267.46      278.38      286.22 : -1            
   170 : 2000-06-18 11:30:00       0     8103       0 :      269.71      279.07      289.11 : -1            
   171 : 2000-06-19 11:30:00       0     8103       0 :      273.82      281.29      290.53 : -1            
   172 : 2000-06-20

  1022 : 2002-10-18 11:30:00       0     8103       0 :      249.53      270.45      282.99 : -1            
  1023 : 2002-10-19 11:30:00       0     8103       0 :      249.59      269.27      281.85 : -1            
  1024 : 2002-10-20 11:30:00       0     8103       0 :      247.20      269.95      282.54 : -1            
  1025 : 2002-10-21 11:30:00       0     8103       0 :      252.95      270.43      282.26 : -1            
  1026 : 2002-10-22 11:30:00       0     8103       0 :      253.96      270.87      286.37 : -1            
  1027 : 2002-10-23 11:30:00       0     8103       0 :      253.40      272.04      283.51 : -1            
  1028 : 2002-10-24 11:30:00       0     8103       0 :      258.00      272.77      282.21 : -1            
  1029 : 2002-10-25 11:30:00       0     8103       0 :      260.14      274.86      284.67 : -1            
  1030 : 2002-10-26 11:30:00       0     8103       0 :      262.98      276.22      284.11 : -1            
  1031 : 2002-10-27

  1827 : 2004-12-31 11:30:00       0     8103       0 :      255.89      272.96      280.69 : -1            
  1828 : 2005-01-01 11:30:00       0     8103       0 :      255.71      271.54      280.56 : -1            
  1829 : 2005-01-02 11:30:00       0     8103       0 :      258.65      272.44      279.70 : -1            
  1830 : 2005-01-03 11:30:00       0     8103       0 :      258.66      272.55      280.45 : -1            
  1831 : 2005-01-04 11:30:00       0     8103       0 :      255.69      271.97      281.34 : -1            
  1832 : 2005-01-05 11:30:00       0     8103       0 :      254.42      271.88      280.68 : -1            
  1833 : 2005-01-06 11:30:00       0     8103       0 :      254.50      271.71      280.72 : -1            
  1834 : 2005-01-07 11:30:00       0     8103       0 :      257.29      272.86      282.35 : -1            
  1835 : 2005-01-08 11:30:00       0     8103       0 :      255.06      273.13      281.49 : -1            
  1836 : 2005-01-09

  2546 : 2006-12-20 11:30:00       0     8103       0 :      248.29      271.51      282.67 : -1            
  2547 : 2006-12-21 11:30:00       0     8103       0 :      259.54      275.23      281.63 : -1            
  2548 : 2006-12-22 11:30:00       0     8103       0 :      260.36      274.94      282.59 : -1            
  2549 : 2006-12-23 11:30:00       0     8103       0 :      262.28      274.55      282.53 : -1            
  2550 : 2006-12-24 11:30:00       0     8103       0 :      259.02      272.55      281.95 : -1            
  2551 : 2006-12-25 11:30:00       0     8103       0 :      253.09      270.46      281.36 : -1            
  2552 : 2006-12-26 11:30:00       0     8103       0 :      255.34      272.01      281.98 : -1            
  2553 : 2006-12-27 11:30:00       0     8103       0 :      262.11      273.76      281.04 : -1            
  2554 : 2006-12-28 11:30:00       0     8103       0 :      259.61      271.90      280.70 : -1            
  2555 : 2006-12-29

  3534 : 2009-09-03 11:30:00       0     8103       0 :      273.33      284.11      290.79 : -1            
  3535 : 2009-09-04 11:30:00       0     8103       0 :      272.86      284.34      290.31 : -1            
  3536 : 2009-09-05 11:30:00       0     8103       0 :      274.32      284.07      289.44 : -1            
  3537 : 2009-09-06 11:30:00       0     8103       0 :      273.41      283.19      289.16 : -1            
  3538 : 2009-09-07 11:30:00       0     8103       0 :      271.77      282.99      290.15 : -1            
  3539 : 2009-09-08 11:30:00       0     8103       0 :      273.74      283.88      290.91 : -1            
  3540 : 2009-09-09 11:30:00       0     8103       0 :      276.15      284.47      291.13 : -1            
  3541 : 2009-09-10 11:30:00       0     8103       0 :      273.62      282.23      290.62 : -1            
  3542 : 2009-09-11 11:30:00       0     8103       0 :      272.11      281.47      290.16 : -1            
  3543 : 2009-09-12

  4510 : 2012-05-06 11:30:00       0     8103       0 :      261.20      275.00      284.65 : -1            
  4511 : 2012-05-07 11:30:00       0     8103       0 :      261.51      274.82      286.51 : -1            
  4512 : 2012-05-08 11:30:00       0     8103       0 :      262.42      274.91      285.27 : -1            
  4513 : 2012-05-09 11:30:00       0     8103       0 :      266.56      276.88      286.65 : -1            
  4514 : 2012-05-10 11:30:00       0     8103       0 :      268.24      278.63      287.42 : -1            
  4515 : 2012-05-11 11:30:00       0     8103       0 :      268.56      278.43      290.55 : -1            
  4516 : 2012-05-12 11:30:00       0     8103       0 :      266.95      275.84      282.57 : -1            
  4517 : 2012-05-13 11:30:00       0     8103       0 :      265.55      274.93      281.92 : -1            
  4518 : 2012-05-14 11:30:00       0     8103       0 :      266.79      276.81      283.11 : -1            
  4519 : 2012-05-15

  5475 : 2014-12-27 11:30:00       0     8103       0 :      246.06      263.74      277.47 : -1            
  5476 : 2014-12-28 11:30:00       0     8103       0 :      243.35      262.27      277.93 : -1            
  5477 : 2014-12-29 11:30:00       0     8103       0 :      242.89      263.48      279.67 : -1            
  5478 : 2014-12-30 11:30:00       0     8103       0 :      248.53      269.12      280.98 : -1            
  5479 : 2014-12-31 11:30:00       0     8103       0 :      254.19      273.50      281.47 : -1            
  5480 : 2015-01-01 11:30:00       0     8103       0 :      263.82      274.82      281.56 : -1            
  5481 : 2015-01-02 11:30:00       0     8103       0 :      262.83      274.45      280.57 : -1            
  5482 : 2015-01-03 11:30:00       0     8103       0 :      261.61      272.33      279.76 : -1            
  5483 : 2015-01-04 11:30:00       0     8103       0 :      252.11      269.45      280.14 : -1            
  5484 : 2015-01-05

  6371 : 2017-06-10 11:30:00       0     8103       0 :      271.66      282.07      287.94 : -1            
  6372 : 2017-06-11 11:30:00       0     8103       0 :      271.62      282.40      289.45 : -1            
  6373 : 2017-06-12 11:30:00       0     8103       0 :      272.09      282.80      288.38 : -1            
  6374 : 2017-06-13 11:30:00       0     8103       0 :      271.52      281.96      287.72 : -1            
  6375 : 2017-06-14 11:30:00       0     8103       0 :      271.76      281.29      289.16 : -1            
  6376 : 2017-06-15 11:30:00       0     8103       0 :      274.43      281.57      289.05 : -1            
  6377 : 2017-06-16 11:30:00       0     8103       0 :      274.54      282.38      289.45 : -1            
  6378 : 2017-06-17 11:30:00       0     8103       0 :      274.07      283.02      290.40 : -1            
  6379 : 2017-06-18 11:30:00       0     8103       0 :      274.51      283.63      290.88 : -1            
  6380 : 2017-06-19

In [12]:
!cdo eca_fd era5-nordics-t2m_k_daily_min.nc era5-nordics-frost-days.nc

cdo eca_fd: Processed 55246254 values from 1 variable over 6818 timesteps [0.78s 47MB]


In [13]:
!cdo info era5-nordics-frost-days.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2018-08-31 11:30:00       0     8103       0 :      29.000      2192.6      4713.0 : -1            
cdo info: Processed 8103 values from 1 variable over 1 timestep [0.00s 45MB]


# Temperature at 2m - Monthly Climatology

In [13]:
!cdo -b 32 ymonmean -addc,-273.15 -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_monthly.nc

cdo ymonmean: Started child process "addc,-273.15 -select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) addc: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe3.1)".
cdo(4) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 35%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 36%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 37%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input gri

cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 70%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 71%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 72%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
 73%cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (Warning): xsize of input grids differ!
cdo(4) cat (Warning): ysize of input grids differ!
cdo(4) cat (War

In [ ]:
!cdo info era5-nordics-t2m_monthly.nc

In [ ]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-t2m_monthly.nc",
    blob_name="era5-nordics-t2m_monthly.nc",
    timeout=180
)

In [15]:
!cdo info era5-nordics-t2m_mean.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2018-01-01 23:00:00       0     8103       0 :     -3.9081      10.837      18.291 : -1            
     2 : 2018-01-02 23:00:00       0     8103       0 :     -2.6651      11.206      18.362 : -1            
     3 : 2018-01-03 23:00:00       0     8103       0 :     -3.2519      11.474      18.202 : -1            
     4 : 2018-01-04 23:00:00       0     8103       0 :     -4.0808      11.490      18.317 : -1            
     5 : 2018-01-05 23:00:00       0     8103       0 :     -3.6812      11.315      18.515 : -1            
     6 : 2018-01-06 23:00:00       0     8103       0 :     -4.2181      11.347      18.350 : -1            
     7 : 2018-01-07 23:00:00       0     8103       0 :     -3.1374      11.794      18.444 : -1            
     8 : 2018-01-08 23:00:00       0     8103       0 :     -3.6237      12.072      18.513 : -1            
     9 : 2018-01-09 2

   197 : 2018-07-15 23:00:00       0     8103       0 :      19.057      24.846      32.288 : -1            
   198 : 2018-07-16 23:00:00       0     8103       0 :      18.993      24.926      32.508 : -1            
   199 : 2018-07-17 23:00:00       0     8103       0 :      18.733      24.902      32.602 : -1            
   200 : 2018-07-18 23:00:00       0     8103       0 :      18.818      24.913      32.338 : -1            
   201 : 2018-07-19 23:00:00       0     8103       0 :      18.884      24.968      32.657 : -1            
   202 : 2018-07-20 23:00:00       0     8103       0 :      18.767      24.809      32.606 : -1            
   203 : 2018-07-21 23:00:00       0     8103       0 :      18.428      24.703      32.652 : -1            
   204 : 2018-07-22 23:00:00       0     8103       0 :      18.692      24.741      32.480 : -1            
   205 : 2018-07-23 23:00:00       0     8103       0 :      18.876      24.727      32.519 : -1            
   206 : 2018-07-24

# Humidity at 2m - Hourly Climatology

In [115]:
!cdo -b 32 yhourmean -expr,'rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15))))' -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-rh_hourly.nc

cdo yhourmean: Started child process "expr,rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15)))) -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) expr: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2639762928 values from 446 variables over 162888 timesteps ( 229.97s )
cdo(2) expr: Processed 2639762928 values from 2 variables over 162888 timesteps ( 229.97s )
cdo yhourmean: Processed 1319881464 values from 1 variable over 162888 timesteps ( 229.97s 780MB )


In [116]:
!cdo info era5-nordics-rh_hourly.nc

    -1 :       Date     Time   Level Gridsize    Miss :     Minimum        Mean     Maximum : Parameter ID
     1 : 2018-01-01 00:00:00       0     8103       0 :      66.965      84.276      95.885 : -1            
     2 : 2018-01-01 01:00:00       0     8103       0 :      66.359      84.323      95.640 : -1            
     3 : 2018-01-01 02:00:00       0     8103       0 :      66.115      84.371      95.375 : -1            
     4 : 2018-01-01 03:00:00       0     8103       0 :      66.061      84.399      95.810 : -1            
     5 : 2018-01-01 04:00:00       0     8103       0 :      65.787      84.410      95.053 : -1            
     6 : 2018-01-01 05:00:00       0     8103       0 :      65.620      84.373      94.917 : -1            
     7 : 2018-01-01 06:00:00       0     8103       0 :      65.900      84.344      95.142 : -1            
     8 : 2018-01-01 07:00:00       0     8103       0 :      65.555      84.331      95.159 : -1            
     9 : 2018-01-01 0

   278 : 2018-01-12 13:00:00       0     8103       0 :      69.740      83.105      92.608 : -1            
   279 : 2018-01-12 14:00:00       0     8103       0 :      69.354      83.483      93.116 : -1            
   280 : 2018-01-12 15:00:00       0     8103       0 :      69.089      83.721      93.403 : -1            
   281 : 2018-01-12 16:00:00       0     8103       0 :      68.225      83.818      93.052 : -1            
   282 : 2018-01-12 17:00:00       0     8103       0 :      69.235      84.057      93.264 : -1            
   283 : 2018-01-12 18:00:00       0     8103       0 :      69.875      84.121      93.482 : -1            
   284 : 2018-01-12 19:00:00       0     8103       0 :      69.372      84.186      93.261 : -1            
   285 : 2018-01-12 20:00:00       0     8103       0 :      69.435      84.276      93.456 : -1            
   286 : 2018-01-12 21:00:00       0     8103       0 :      69.468      84.260      93.831 : -1            
   287 : 2018-01-12

   569 : 2018-01-24 16:00:00       0     8103       0 :      68.474      83.063      93.013 : -1            
   570 : 2018-01-24 17:00:00       0     8103       0 :      69.126      83.366      93.188 : -1            
   571 : 2018-01-24 18:00:00       0     8103       0 :      70.088      83.491      93.115 : -1            
   572 : 2018-01-24 19:00:00       0     8103       0 :      69.741      83.593      93.373 : -1            
   573 : 2018-01-24 20:00:00       0     8103       0 :      69.538      83.684      93.787 : -1            
   574 : 2018-01-24 21:00:00       0     8103       0 :      69.890      83.716      93.955 : -1            
   575 : 2018-01-24 22:00:00       0     8103       0 :      68.711      83.765      94.578 : -1            
   576 : 2018-01-24 23:00:00       0     8103       0 :      68.230      83.774      94.887 : -1            
   577 : 2018-01-25 00:00:00       0     8103       0 :      67.749      83.801      94.961 : -1            
   578 : 2018-01-25

   780 : 2018-02-02 11:00:00       0     8103       0 :      68.810      82.102      91.975 : -1            
   781 : 2018-02-02 12:00:00       0     8103       0 :      68.382      81.765      91.790 : -1            
   782 : 2018-02-02 13:00:00       0     8103       0 :      68.390      81.692      91.756 : -1            
   783 : 2018-02-02 14:00:00       0     8103       0 :      68.694      82.073      92.802 : -1            
   784 : 2018-02-02 15:00:00       0     8103       0 :      68.405      82.406      93.968 : -1            
   785 : 2018-02-02 16:00:00       0     8103       0 :      68.977      82.687      94.939 : -1            
   786 : 2018-02-02 17:00:00       0     8103       0 :      69.254      83.120      95.026 : -1            
   787 : 2018-02-02 18:00:00       0     8103       0 :      69.307      83.251      94.790 : -1            
   788 : 2018-02-02 19:00:00       0     8103       0 :      69.541      83.344      94.581 : -1            
   789 : 2018-02-02

  1046 : 2018-02-13 13:00:00       0     8103       0 :      65.532      80.151      90.509 : -1            
  1047 : 2018-02-13 14:00:00       0     8103       0 :      66.699      80.477      91.161 : -1            
  1048 : 2018-02-13 15:00:00       0     8103       0 :      66.092      81.115      91.157 : -1            
  1049 : 2018-02-13 16:00:00       0     8103       0 :      66.041      81.652      90.994 : -1            
  1050 : 2018-02-13 17:00:00       0     8103       0 :      65.293      82.417      92.260 : -1            
  1051 : 2018-02-13 18:00:00       0     8103       0 :      64.900      82.628      92.665 : -1            
  1052 : 2018-02-13 19:00:00       0     8103       0 :      65.002      82.744      93.143 : -1            
  1053 : 2018-02-13 20:00:00       0     8103       0 :      65.556      82.897      93.382 : -1            
  1054 : 2018-02-13 21:00:00       0     8103       0 :      65.521      82.928      93.905 : -1            
  1055 : 2018-02-13

...

  6610 : 2017-10-02 09:00:00       0     8103       0 :      68.014      82.106      92.175 : -1            
  6611 : 2017-10-02 10:00:00       0     8103       0 :      57.908      78.737      89.438 : -1            
  6612 : 2017-10-02 11:00:00       0     8103       0 :      57.744      77.605      88.422 : -1            
  6613 : 2017-10-02 12:00:00       0     8103       0 :      57.923      76.804      88.340 : -1            
  6614 : 2017-10-02 13:00:00       0     8103       0 :      58.334      76.422      88.251 : -1            
  6615 : 2017-10-02 14:00:00       0     8103       0 :      61.462      77.255      88.686 : -1            
  6616 : 2017-10-02 15:00:00       0     8103       0 :      65.977      78.490      89.205 : -1            
  6617 : 2017-10-02 16:00:00       0     8103       0 :      66.483      79.763      90.460 : -1            
  6618 : 2017-10-02 17:00:00       0     8103       0 :      71.703      83.024      93.577 : -1            
  6619 : 2017-10-02

  7124 : 2017-10-23 19:00:00       0     8103       0 :      70.426      83.931      94.118 : -1            
  7125 : 2017-10-23 20:00:00       0     8103       0 :      70.093      84.583      94.960 : -1            
  7126 : 2017-10-23 21:00:00       0     8103       0 :      69.688      84.865      95.258 : -1            
  7127 : 2017-10-23 22:00:00       0     8103       0 :      69.910      85.337      95.571 : -1            
  7128 : 2017-10-23 23:00:00       0     8103       0 :      70.322      85.497      95.751 : -1            
  7129 : 2017-10-24 00:00:00       0     8103       0 :      70.738      85.647      96.005 : -1            
  7130 : 2017-10-24 01:00:00       0     8103       0 :      71.601      85.756      95.876 : -1            
  7131 : 2017-10-24 02:00:00       0     8103       0 :      71.604      85.978      95.910 : -1            
  7132 : 2017-10-24 03:00:00       0     8103       0 :      71.496      86.037      96.316 : -1            
  7133 : 2017-10-24

  7636 : 2017-11-14 03:00:00       0     8103       0 :      68.804      85.604      96.168 : -1            
  7637 : 2017-11-14 04:00:00       0     8103       0 :      68.778      85.637      96.388 : -1            
  7638 : 2017-11-14 05:00:00       0     8103       0 :      69.280      85.693      96.209 : -1            
  7639 : 2017-11-14 06:00:00       0     8103       0 :      68.931      85.682      96.010 : -1            
  7640 : 2017-11-14 07:00:00       0     8103       0 :      68.786      85.669      96.102 : -1            
  7641 : 2017-11-14 08:00:00       0     8103       0 :      68.446      85.252      95.464 : -1            
  7642 : 2017-11-14 09:00:00       0     8103       0 :      68.121      84.936      94.866 : -1            
  7643 : 2017-11-14 10:00:00       0     8103       0 :      68.188      83.916      93.196 : -1            
  7644 : 2017-11-14 11:00:00       0     8103       0 :      68.374      83.589      93.063 : -1            
  7645 : 2017-11-14

  8094 : 2017-12-03 05:00:00       0     8103       0 :      68.170      85.589      95.866 : -1            
  8095 : 2017-12-03 06:00:00       0     8103       0 :      68.762      85.591      95.819 : -1            
  8096 : 2017-12-03 07:00:00       0     8103       0 :      69.082      85.581      95.848 : -1            
  8097 : 2017-12-03 08:00:00       0     8103       0 :      69.287      85.365      95.888 : -1            
  8098 : 2017-12-03 09:00:00       0     8103       0 :      69.474      85.099      95.450 : -1            
  8099 : 2017-12-03 10:00:00       0     8103       0 :      69.765      84.573      94.993 : -1            
  8100 : 2017-12-03 11:00:00       0     8103       0 :      69.549      84.302      94.834 : -1            
  8101 : 2017-12-03 12:00:00       0     8103       0 :      70.321      84.158      94.852 : -1            
  8102 : 2017-12-03 13:00:00       0     8103       0 :      71.271      84.207      94.778 : -1            
  8103 : 2017-12-03

  8616 : 2017-12-24 23:00:00       0     8103       0 :      69.808      83.899      95.156 : -1            
  8617 : 2017-12-25 00:00:00       0     8103       0 :      69.701      83.961      95.078 : -1            
  8618 : 2017-12-25 01:00:00       0     8103       0 :      68.740      83.988      94.896 : -1            
  8619 : 2017-12-25 02:00:00       0     8103       0 :      68.324      84.027      94.496 : -1            
  8620 : 2017-12-25 03:00:00       0     8103       0 :      68.713      84.061      94.326 : -1            
  8621 : 2017-12-25 04:00:00       0     8103       0 :      68.728      83.997      93.993 : -1            
  8622 : 2017-12-25 05:00:00       0     8103       0 :      68.533      83.997      94.094 : -1            
  8623 : 2017-12-25 06:00:00       0     8103       0 :      68.876      83.941      94.172 : -1            
  8624 : 2017-12-25 07:00:00       0     8103       0 :      67.935      83.883      94.149 : -1            
  8625 : 2017-12-25

In [119]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-rh_hourly.nc",
    blob_name="era5-nordics-rh_hourly.nc",
    timeout=180
)

# Humidity at 2m - Daily Climatology

In [117]:
!cdo -b 32 ydaymean -expr,'rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15))))' -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-rh_daily.nc

cdo ydaymean: Started child process "expr,rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15)))) -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) expr: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2639762928 values from 446 variables over 162888 timesteps ( 271.72s )
cdo(2) expr: Processed 2639762928 values from 2 variables over 162888 timesteps ( 271.72s )
cdo ydaymean: Processed 1319881464 values from 1 variable over 162888 timesteps ( 271.72s 365MB )


In [118]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-rh_daily.nc",
    blob_name="era5-nordics-rh_daily.nc",
    timeout=180
)

# Humidity at 2m - Monthly Climatology

In [120]:
!cdo -b 32 ydaymean -expr,'rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15))))' -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-rh_monthly.nc

cdo ydaymean: Started child process "expr,rh=100*(exp((17.625*(d2m-273.15))/(243.04+(d2m-273.15)))/exp((17.625*(t2m-273.15))/(243.04+(t2m-273.15)))) -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) expr: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2639762928 values from 446 variables over 162888 timesteps ( 234.84s )
cdo(2) expr: Processed 2639762928 values from 2 variables over 162888 timesteps ( 234.84s )
cdo ydaymean: Processed 1319881464 values from 1 variable over 162888 timesteps ( 234.84s 206MB )


In [121]:
blob_storage.create_blob_from_path(
    container_name=blob_storage_container_name,
    file_path="era5-nordics-rh_monthly.nc",
    blob_name="era5-nordics-rh_monthly.nc",
    timeout=180
)

# Temperature at 2m (K) - Daily Summaries

Average/Mean temperatures

In [2]:
!cdo -b 32 daymean -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_daily_mean_k.nc

cdo daymean: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2651820192 values from 448 variables over 163632 timesteps ( 36.48s )
cdo(2) select: Processed 1325910096 values from 2 variables over 163632 timesteps ( 36.49s )
cdo daymean: Processed 1325910096 values from 1 variable over 163632 timesteps ( 36.49s 14MB )


Daily maximum temperatures

In [6]:
!cdo -b 32 daymax -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_daily_max_k.nc

cdo daymax: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2651820192 values from 448 variables over 163632 timesteps ( 38.74s )
cdo(2) select: Processed 1325910096 values from 2 variables over 163632 timesteps ( 38.74s )
cdo daymax: Processed 1325910096 values from 1 variable over 163632 timesteps ( 38.74s 14MB )


Daily minimum temperatures

In [7]:
!cdo -b 32 daymin -select,name=t2m -cat 'era5-nordics-temperature-*-*.nc' era5-nordics-t2m_daily_min_k.nc

cdo daymin: Started child process "select,name=t2m -cat era5-nordics-temperature-*-*.nc (pipe1.1)".
cdo(2) select: Started child process "cat era5-nordics-temperature-*-*.nc (pipe2.1)".
cdo(3) cat:                        1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 9 9 9 9 9 9 9 9 9 910cdo(3) cat: Processed 2651820192 values from 448 variables over 163632 timesteps ( 38.97s )
cdo(2) select: Processed 1325910096 values from 2 variables over 163632 timesteps ( 38.98s )
cdo daymin: Processed 1325910096 values from 1 variable over 163632 timesteps ( 38.98s 14MB )
